In [1]:
import numpy as np
import scipy
import implicit
import pandas as pd
import os

# 1) 데이터 준비와 전처리

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


# 2) 분석해 봅시다.

In [19]:
data = pd.merge(ratings, movies, on = 'movie_id')
data['title'] = data['title'].str.lower() # 검색을 쉽게 하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
data = data[['user_id', 'title', 'counts']]

data.head(10)

,user_id,title,counts
0,1,one flew over the cuckoo's nest (1975),5
1,2,one flew over the cuckoo's nest (1975),5
2,12,one flew over the cuckoo's nest (1975),4
3,15,one flew over the cuckoo's nest (1975),4
4,17,one flew over the cuckoo's nest (1975),5
5,18,one flew over the cuckoo's nest (1975),4
6,19,one flew over the cuckoo's nest (1975),5
7,24,one flew over the cuckoo's nest (1975),5
8,28,one flew over the cuckoo's nest (1975),3
9,33,one flew over the cuckoo's nest (1975),5


In [21]:
print('ratings에 있는 유니크한 영화 개수:', data['title'].nunique())
print('ratings에 있는 유니크한 사용자 수:', data['user_id'].nunique())
print('가장 인기 있는 영화 30개(인기순):')

movie_count = data.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

ratings에 있는 유니크한 영화 개수: 3628
ratings에 있는 유니크한 사용자 수: 6039
가장 인기 있는 영화 30개(인기순):


title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

# 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [22]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['terminator, the (1984)' , 'forrest gump (1994)' ,'pulp fiction (1994)' ,'men in black (1997)' ,'ghostbusters (1984)']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['taejune']*5, 'title': my_favorite, 'counts':[4]*5})

if not data.isin({'user_id':['taejune']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,title,counts
836473,5851,one little indian (1973),5
836474,5854,slaughterhouse (1987),4
836475,5854,"promise, the (versprechen, das) (1994)",3
836476,5938,"five wives, three secretaries and me (1998)",4
836477,5948,identification of a woman (identificazione di ...,5
0,taejune,"terminator, the (1984)",4
1,taejune,forrest gump (1994),4
2,taejune,pulp fiction (1994),4
3,taejune,men in black (1997),4
4,taejune,ghostbusters (1984),4


In [24]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [42]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['taejune'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(title_to_idx['men in black (1997)'])

6039
175


# 4) CSR matrix를 직접 만들어 봅시다.

In [28]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# title_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,counts
0,0,0,5
1,1,0,5
2,2,0,4
3,3,0,4
4,4,0,5
...,...,...,...
0,6039,200,4
1,6039,160,4
2,6039,222,4
3,6039,175,4


In [30]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_title = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape= (num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [31]:
from implicit.als import AlternatingLeastSquares

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [32]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [33]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [34]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [44]:
taejune, men_in_black = user_to_idx['taejune'], title_to_idx['men in black (1997)']
taejune_vector, men_in_black_vector = als_model.user_factors[taejune], als_model.item_factors[men_in_black]

In [46]:
taejune_vector

array([-5.90279222e-01, -2.41623476e-01,  2.04140589e-01, -2.12962806e-01,
       -1.87580854e-01,  2.44142815e-01, -8.39553118e-01, -1.17611301e+00,
        2.14782536e-01, -5.40048838e-01,  8.87865648e-02, -4.74711001e-01,
        4.23215151e-01,  4.44868684e-01,  5.46762347e-01, -6.68445081e-02,
        8.06423426e-01,  4.37483311e-01,  2.02229545e-01, -1.60743997e-01,
       -4.71761525e-02,  7.41731286e-01, -3.42278570e-01, -3.30546784e-04,
       -2.75987715e-01,  2.31685400e-01,  4.36832190e-01,  1.56327114e-01,
       -7.69263446e-01, -7.30994791e-02,  2.33176425e-01, -1.25289649e-01,
        2.98411667e-01,  1.09158322e-01,  4.43031818e-01,  2.76466042e-01,
        1.54115170e-01, -1.63193271e-01, -5.41687384e-02, -9.54016224e-02,
       -9.44080651e-02,  3.76369447e-01, -5.68130851e-01, -2.11442485e-02,
       -3.11470479e-01,  1.55292347e-01, -8.01512957e-01,  3.63124639e-01,
       -1.35560855e-01, -6.78580999e-01, -1.30805239e-01, -1.88624829e-01,
       -5.29110670e-01, -

In [47]:
men_in_black_vector

array([-0.00842962,  0.00749896,  0.0127876 ,  0.00863832, -0.01035982,
        0.00316326, -0.02141629, -0.03852015,  0.0158637 ,  0.03507251,
        0.01426642, -0.02438173, -0.01914337, -0.01238089, -0.00035474,
        0.01536127,  0.01954706,  0.01550917,  0.00046999,  0.00211523,
       -0.00267011,  0.03273964, -0.02251273,  0.02123083, -0.02188584,
        0.0154984 ,  0.02983838,  0.01763257,  0.00418592,  0.01119256,
        0.01407741, -0.00525589, -0.01346784,  0.00366196, -0.00460149,
       -0.00234894,  0.02322184,  0.01485264, -0.01032876,  0.00804107,
        0.01911635, -0.02414637,  0.0079264 , -0.0059435 ,  0.01444525,
        0.01682318, -0.00542586,  0.00896003,  0.02508112, -0.00351585,
       -0.03119492,  0.0142193 , -0.02463176,  0.03462857,  0.03365576,
        0.01727883,  0.01280033,  0.03820498, -0.0084773 ,  0.01281677,
       -0.00168739,  0.01845865, -0.01288867,  0.01865461,  0.03422838,
        0.04821969,  0.03090717,  0.00042779, -0.01654241, -0.00

In [49]:
np.dot(taejune_vector, men_in_black_vector)

0.38726166

# 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [51]:
gump = title_to_idx['forrest gump (1994)'] # 선호하는 영화
ryan = title_to_idx['saving private ryan (1998)'] # 그 외의 영화
gump_vector = als_model.item_factors[gump]
np.dot(taejune_vector, gump_vector)

0.46172488

In [52]:
ryan_vector = als_model.item_factors[ryan]
np.dot(taejune_vector, ryan_vector)

0.1430613

# 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [54]:
favorite_movie = 'forrest gump (1994)'
movie_id = title_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(160, 1.0),
 (110, 0.60975283),
 (336, 0.4723566),
 (154, 0.4684204),
 (666, 0.4547838),
 (508, 0.4155792),
 (204, 0.40943053),
 (613, 0.40524912),
 (626, 0.40390596),
 (3614, 0.40164578),
 (105, 0.38630074),
 (340, 0.38499424),
 (1151, 0.38162053),
 (40, 0.3786391),
 (466, 0.37646574)]

In [56]:
# title_to_idx 를 뒤집어, index로부터 title 이름을 얻는 dict를 생성합니다. 
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_movie]

['forrest gump (1994)',
 'groundhog day (1993)',
 'pretty woman (1990)',
 'as good as it gets (1997)',
 'sleepless in seattle (1993)',
 'ghost (1990)',
 'clueless (1995)',
 'four weddings and a funeral (1994)',
 "you've got mail (1998)",
 'outside ozona (1998)',
 'last of the mohicans, the (1992)',
 "my best friend's wedding (1997)",
 'notting hill (1999)',
 'toy story (1995)',
 'wedding singer, the (1998)']

In [57]:
def get_similar_movie(movie_name: str):
    movie_id = title_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_title[i[0]] for i in similar_movie]
    return similar_movie

In [59]:
get_similar_movie('pulp fiction (1994)')

['pulp fiction (1994)',
 'goodfellas (1990)',
 'fargo (1996)',
 'shawshank redemption, the (1994)',
 'usual suspects, the (1995)',
 'reservoir dogs (1992)',
 'silence of the lambs, the (1991)',
 'good will hunting (1997)',
 "schindler's list (1993)",
 'l.a. confidential (1997)']

# 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [60]:
user = user_to_idx['taejune']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(92, 0.40228856),
 (64, 0.3721132),
 (22, 0.34410214),
 (124, 0.3373537),
 (107, 0.33684248),
 (38, 0.3248741),
 (121, 0.31338334),
 (117, 0.30160677),
 (110, 0.269529),
 (51, 0.2678468),
 (233, 0.26138818),
 (44, 0.25854763),
 (62, 0.24250963),
 (224, 0.23898047),
 (99, 0.23773935),
 (120, 0.20859253),
 (157, 0.20102072),
 (201, 0.19559702),
 (15, 0.19519731),
 (637, 0.19363217)]

In [61]:
[idx_to_title[i[0]] for i in movie_recommended]

['terminator 2: judgment day (1991)',
 'star wars: episode vi - return of the jedi (1983)',
 'back to the future (1985)',
 'matrix, the (1999)',
 'jurassic park (1993)',
 'sixth sense, the (1999)',
 'silence of the lambs, the (1991)',
 'star wars: episode v - the empire strikes back (1980)',
 'groundhog day (1993)',
 'fargo (1996)',
 'usual suspects, the (1995)',
 'star wars: episode iv - a new hope (1977)',
 'total recall (1990)',
 'l.a. confidential (1997)',
 'american beauty (1999)',
 'raiders of the lost ark (1981)',
 'shawshank redemption, the (1994)',
 'who framed roger rabbit? (1988)',
 'airplane! (1980)',
 'abyss, the (1989)']

In [64]:
future = title_to_idx['back to the future (1985)']
explain = als_model.explain(user, csr_data, itemid=future)

In [65]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('ghostbusters (1984)', 0.15854607216511532),
 ('forrest gump (1994)', 0.09383121833974134),
 ('terminator, the (1984)', 0.06445744784189897),
 ('men in black (1997)', 0.05108565088302294),
 ('pulp fiction (1994)', -0.029015225733834676)]